In [ ]:
import os
from rdkit import Chem
from rdkit.Chem import AllChem
#import pybel
import subprocess

# Convert MOL to PDBQT with Open Babel using command line
def convert_mol_to_pdbqt(input_path, output_path):
    subprocess.run(["obabel", input_path, "-O", output_path, "--partialcharge", "gasteiger"])

# Define paths
ligand_directory = "/home/ms/Documents/vina_docking/input_ligand"  # Directory containing .sdf ligand files
output_directory = "/home/ms/Documents/vina_docking/output_docking"    # Directory for prepared ligands and docking results
config_path = "/home/ms/Documents/vina_docking/conf.txt"  # Vina configuration file path
vina_path = "/home/ms/Documents/vina_docking/vina"     # config.txt file path

# Ensure output directory exists
os.makedirs(output_directory, exist_ok=True)

# Loop over all .sdf ligand files in the directory
for ligand_file in os.listdir(ligand_directory):
    if ligand_file.endswith(".sdf"):
        ligand_path = os.path.join(ligand_directory, ligand_file)
        
        # Read each molecule in the .sdf file (in case it contains multiple ligands)
        suppl = Chem.SDMolSupplier(ligand_path)
        for idx, mol in enumerate(suppl):
            if mol is None:
                continue  # Skip if there's an issue with the molecule

            # Add hydrogens, generate 3D coordinates, and optimize
            mol = Chem.AddHs(mol)
            AllChem.EmbedMolecule(mol)
            AllChem.UFFOptimizeMolecule(mol)

            #Define paths and perform the conversion
            mol_temp_path = os.path.join(output_directory, f"{ligand_file.replace('.sdf', '')}_mol_{idx}.mol")
            Chem.MolToMolFile(mol, mol_temp_path)

            # Use the function to convert the MOL file to PDBQT
            ligand_prepared_path = mol_temp_path.replace(".mol", ".pdbqt")
            convert_mol_to_pdbqt(mol_temp_path, ligand_prepared_path)

            # Define AutoDock Vina output file path
            docking_output_path = ligand_prepared_path.replace(".pdbqt", "_docked.pdbqt")

            # Run AutoDock Vina
            
            # Construct the vina command with the full path to the executable
            vina_command = f"{vina_path} --config {config_path} --ligand {ligand_prepared_path} --out {docking_output_path}"
            subprocess.run(vina_command, shell=True)

            print(f"Processed and docked ligand: {ligand_file}, molecule index: {idx}")
